In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
from PIL import Image

In [3]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution1D, Convolution2D, ZeroPadding2D, MaxPooling2D

In [4]:
import os

In [5]:
trial_model=load_model('VGGFace.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








C:\Users\Om\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output

In [7]:
def feature_extractor(inpDir,personID,wordID,timestep,ex_type=True): #access directory and extract face vector
    
    
    if ex_type == True:
        Y_train_labels=np.ones((10,512))
    else:
        Y_train_labels=np.zeros((10,512))  #remove after training stage is completed
        
        
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    inpDir=inpDir #input dataset
    linpDir = os.listdir(inpDir) #list all directories in dataset
    #print(linpDir)
    personStr= linpDir[personID]
    personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
    lpersonFolder = os.listdir(personFolder)
    wordID= lpersonFolder[wordID]
    wordFolder = '%s\\%s' % (personFolder,wordID)
    lwordFolder = os.listdir(wordFolder)
    
   
    i=0
    for utterance in lwordFolder:
        
        utterFolder= '%s\\%s' % (wordFolder,utterance)
        lutterFolder = os.listdir(utterFolder)
        colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
        lcolourFolder = os.listdir(colourFolder)
        frame = lcolourFolder[timestep]
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)
            
    print("Extraction completed %d"%i)
    
    WB_CNN= Sequential()
    WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation= 'relu', input_shape=(1,1,2622)))
    WB_CNN.add( Dropout(0.2) )
    WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation='relu',input_shape=(1,1,2622)))
    WB_CNN.add( Dropout(0.2) )
    WB_CNN.add(Convolution2D(512, kernel_size=(1,1), activation='relu'))
    WB_CNN.add(Flatten())   

    WB_CNN.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    WB_CNN.fit(feature_sequence,Y_train_labels,epochs=20,batch_size=2)
    CNN_out = WB_CNN.predict(feature_sequence)
    
    print("Prediction done")
    print(CNN_out.shape)
    return CNN_out   

In [8]:
#for i in range(20):
    #test_run= feature_extractor('C:\\Users\\Om\\Desktop\\Dataset',0,1,i,ex_type=True)
    #print(i)
    

In [9]:
def per_to_feat(inpDir,personID,wordID,ex_type):
    timesteps=20
    count=0
    for i in range(timesteps):
        CNN_out=feature_extractor(inpDir,personID,wordID,i,ex_type)
        count = count + 1
        if count==1 :
            CNN_out_total= CNN_out #maintaining same dimensions
        else :
            CNN_out_total= np.concatenate((CNN_out_total,CNN_out))
        print(str(i) + 'timestep')
    
    print('Concatenation completed')
    return CNN_out_total      

In [10]:
#test_run=per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',0,1,True)

In [11]:
def reshape_LSTM(sample_mat):
    feature_vector=[]
    feature_mat=[]
    count_i=0
    for i in range(10):
        count_j=0
        for j in range(0,200,10):
            j=j+i
            sample_mat_j=sample_mat[j]
            sample_mat_j=sample_mat_j.reshape((1,512))
            if count_j==0:
                feature_vector=sample_mat_j
            else:
                feature_vector=np.concatenate((feature_vector,sample_mat_j))
            count_j = count_j + 1
        
        if count_i==0:
            feature_mat=feature_vector
        else:
            feature_mat= np.concatenate((feature_mat,feature_vector))
        count_i= count_i + 1
    LSTM_input= feature_mat
    return LSTM_input

In [12]:
def person_extractor(personID):
    p_dataset=[]
    count=0
    for i in range(5):
        count = count +1
        if personID==0 and i==0:
            CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',personID,i,True)
        else:
            CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',personID,i,False)
        if count==1 :
            p_dataset = CNN_out_total #done because of need of same dim. for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        print(str(i) + 'word')
    print('----------------')
    print(count)
    print(p_dataset.shape)
    return p_dataset 

In [13]:
#p1_dataset = person_extractor(0)
def person_extractor(personID):
    p_dataset=[]
    count=0
    for i in range(5):
        count = count +1
        if personID==0 and i==0:
            CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',personID,i,True)
        else:
            CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',personID,i,False)
        if count==1 :
            p_dataset = CNN_out_total #done because of need of same dim. for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        print(str(i) + 'word')
    print('----------------')
    print(count)
    print(p_dataset.shape)
    return p_dataset 

In [14]:
p5_f5=person_extractor(4)

Extraction completed 10


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
10/10 [==============================] - 2s 243ms/step - loss: 4.6109 - acc: 0.6639
Epoch 2/20
10/10 [==============================] - 1s 58ms/step - loss: 2.9194 - acc: 0.8074
Epoch 3/20
10/10 [==============================] - 1s 86ms/step - loss: 2.2123 - acc: 0.8574
Epoch 4/20
10/10 [==============================] - 1s 68ms/step - loss: 1.7314 - acc: 0.8887
Epoch 5/20
10/10 [==============================] - 1s 68ms/step - loss: 1.5759 - acc: 0.8992
Epoch 6/20
10/10 [==============================] - 1s 86ms/step - loss: 1.4172 - acc: 0.9100
Epoch 7/20
10/10 [==============================] - 1s 78ms/step - loss: 1.2825 - acc: 0.9186
Epoch 8/20
10/10 [==============================] - 1s 78ms/step - loss: 1.1543 - acc: 0.9270
Epoch 9/20
10/10 [==============================] - 1s 82ms/step - loss: 1.0474 - acc: 0.9336
Epoch 10/20
10/10 [============

10/10 [==============================] - 1s 85ms/step - loss: 0.7510 - acc: 0.9527
Epoch 15/20
10/10 [==============================] - 1s 83ms/step - loss: 0.7193 - acc: 0.9549: 0s - loss: 0.7084 - acc: 0.9
Epoch 16/20
10/10 [==============================] - 1s 74ms/step - loss: 0.7351 - acc: 0.9537
Epoch 17/20
10/10 [==============================] - 1s 56ms/step - loss: 0.6615 - acc: 0.9578
Epoch 18/20
10/10 [==============================] - 1s 64ms/step - loss: 0.6211 - acc: 0.9604
Epoch 19/20
10/10 [==============================] - 1s 64ms/step - loss: 0.5638 - acc: 0.9645
Epoch 20/20
10/10 [==============================] - 1s 57ms/step - loss: 0.5832 - acc: 0.9631
Prediction done
(10, 512)
3timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 3s 267ms/step - loss: 4.4720 - acc: 0.6701
Epoch 2/20
10/10 [==============================] - 1s 51ms/step - loss: 2.7941 - acc: 0.8125
Epoch 3/20
10/10 [==============================] - 1s 62ms/step - l

10/10 [==============================] - 1s 53ms/step - loss: 0.5956 - acc: 0.9625
Epoch 18/20
10/10 [==============================] - 1s 65ms/step - loss: 0.5831 - acc: 0.9631
Epoch 19/20
10/10 [==============================] - 1s 58ms/step - loss: 0.5549 - acc: 0.9650
Epoch 20/20
10/10 [==============================] - 1s 71ms/step - loss: 0.5433 - acc: 0.9654
Prediction done
(10, 512)
7timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 2s 225ms/step - loss: 4.1400 - acc: 0.6957
Epoch 2/20
10/10 [==============================] - 0s 45ms/step - loss: 2.8696 - acc: 0.8090
Epoch 3/20
10/10 [==============================] - 0s 46ms/step - loss: 1.8722 - acc: 0.8766
Epoch 4/20
10/10 [==============================] - 0s 45ms/step - loss: 1.4439 - acc: 0.9070
Epoch 5/20
10/10 [==============================] - 0s 45ms/step - loss: 1.1443 - acc: 0.9260
Epoch 6/20
10/10 [==============================] - 0s 43ms/step - loss: 0.9807 - acc: 0.9369
Epoch 7

10/10 [==============================] - 0s 47ms/step - loss: 0.1310 - acc: 0.9916
Epoch 20/20
10/10 [==============================] - 0s 45ms/step - loss: 0.1187 - acc: 0.9924
Prediction done
(10, 512)
11timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 2s 204ms/step - loss: 3.7097 - acc: 0.7064
Epoch 2/20
10/10 [==============================] - 0s 43ms/step - loss: 1.9472 - acc: 0.8578
Epoch 3/20
10/10 [==============================] - 0s 44ms/step - loss: 0.9745 - acc: 0.9322
Epoch 4/20
10/10 [==============================] - 0s 45ms/step - loss: 0.6208 - acc: 0.9582
Epoch 5/20
10/10 [==============================] - 0s 45ms/step - loss: 0.5093 - acc: 0.9670
Epoch 6/20
10/10 [==============================] - 0s 45ms/step - loss: 0.4316 - acc: 0.9719
Epoch 7/20
10/10 [==============================] - 0s 46ms/step - loss: 0.3747 - acc: 0.9760
Epoch 8/20
10/10 [==============================] - 0s 46ms/step - loss: 0.3837 - acc: 0.9756
Epoch 9/

10/10 [==============================] - 2s 234ms/step - loss: 0.8431 - acc: 0.8699
Epoch 2/20
10/10 [==============================] - 0s 45ms/step - loss: 0.3382 - acc: 0.9615
Epoch 3/20
10/10 [==============================] - 0s 45ms/step - loss: 0.1128 - acc: 0.9883
Epoch 4/20
10/10 [==============================] - 0s 45ms/step - loss: 0.0630 - acc: 0.9945
Epoch 5/20
10/10 [==============================] - 0s 47ms/step - loss: 0.0410 - acc: 0.9975
Epoch 6/20
10/10 [==============================] - 0s 43ms/step - loss: 0.0321 - acc: 0.9975
Epoch 7/20
10/10 [==============================] - 0s 47ms/step - loss: 0.0348 - acc: 0.9975
Epoch 8/20
10/10 [==============================] - 0s 47ms/step - loss: 0.0412 - acc: 0.9971
Epoch 9/20
10/10 [==============================] - 0s 47ms/step - loss: 0.0314 - acc: 0.9979
Epoch 10/20
10/10 [==============================] - 0s 45ms/step - loss: 0.0320 - acc: 0.9977
Epoch 11/20
10/10 [==============================] - 0s 45ms/step - l

...

10/10 [==============================] - 0s 49ms/step - loss: 0.7834 - acc: 0.9500
Epoch 10/20
10/10 [==============================] - 0s 48ms/step - loss: 0.6985 - acc: 0.9557
Epoch 11/20
10/10 [==============================] - 0s 48ms/step - loss: 0.6700 - acc: 0.9576
Epoch 12/20
10/10 [==============================] - 0s 47ms/step - loss: 0.6681 - acc: 0.9574
Epoch 13/20
10/10 [==============================] - 0s 46ms/step - loss: 0.5888 - acc: 0.9629
Epoch 14/20
10/10 [==============================] - 0s 48ms/step - loss: 0.5922 - acc: 0.9625
Epoch 15/20
10/10 [==============================] - 0s 48ms/step - loss: 0.5765 - acc: 0.9637
Epoch 16/20
10/10 [==============================] - 1s 50ms/step - loss: 0.5919 - acc: 0.9627
Epoch 17/20
10/10 [==============================] - 0s 49ms/step - loss: 0.5513 - acc: 0.9654
Epoch 18/20
10/10 [==============================] - 0s 45ms/step - loss: 0.5700 - acc: 0.9641
Epoch 19/20
10/10 [==============================] - 0s 45ms/s

10/10 [==============================] - 0s 50ms/step - loss: 0.6540 - acc: 0.9590
Epoch 13/20
10/10 [==============================] - 1s 52ms/step - loss: 0.6636 - acc: 0.9582
Epoch 14/20
10/10 [==============================] - 1s 50ms/step - loss: 0.6510 - acc: 0.9590
Epoch 15/20
10/10 [==============================] - 0s 48ms/step - loss: 0.6386 - acc: 0.9598
Epoch 16/20
10/10 [==============================] - 1s 51ms/step - loss: 0.6199 - acc: 0.9609
Epoch 17/20
10/10 [==============================] - 0s 50ms/step - loss: 0.6075 - acc: 0.9617
Epoch 18/20
10/10 [==============================] - 1s 50ms/step - loss: 0.5515 - acc: 0.9652
Epoch 19/20
10/10 [==============================] - 0s 47ms/step - loss: 0.5268 - acc: 0.9668
Epoch 20/20
10/10 [==============================] - 1s 52ms/step - loss: 0.5294 - acc: 0.9668
Prediction done
(10, 512)
6timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 9s 945ms/step - loss: 4.5568 - acc: 0.6693
Ep

10/10 [==============================] - 0s 50ms/step - loss: 0.7745 - acc: 0.9510
Epoch 16/20
10/10 [==============================] - 0s 48ms/step - loss: 0.7605 - acc: 0.9518
Epoch 17/20
10/10 [==============================] - 0s 48ms/step - loss: 0.7074 - acc: 0.9553
Epoch 18/20
10/10 [==============================] - 0s 50ms/step - loss: 0.7208 - acc: 0.9539
Epoch 19/20
10/10 [==============================] - 0s 48ms/step - loss: 0.6560 - acc: 0.9582
Epoch 20/20
10/10 [==============================] - 0s 48ms/step - loss: 0.6485 - acc: 0.9588
Prediction done
(10, 512)
10timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 10s 990ms/step - loss: 4.4821 - acc: 0.6689
Epoch 2/20
10/10 [==============================] - 0s 47ms/step - loss: 2.7547 - acc: 0.8094
Epoch 3/20
10/10 [==============================] - 0s 45ms/step - loss: 1.8727 - acc: 0.8744
Epoch 4/20
10/10 [==============================] - 0s 48ms/step - loss: 1.2201 - acc: 0.9197
Epo

10/10 [==============================] - 0s 50ms/step - loss: 0.0031 - acc: 0.9998
Epoch 19/20
10/10 [==============================] - 1s 50ms/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 20/20
10/10 [==============================] - 0s 50ms/step - loss: 0.0066 - acc: 0.9994
Prediction done
(10, 512)
14timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 11s 1s/step - loss: 0.9598 - acc: 0.8525
Epoch 2/20
10/10 [==============================] - 0s 48ms/step - loss: 0.3284 - acc: 0.9623
Epoch 3/20
10/10 [==============================] - 0s 48ms/step - loss: 0.1232 - acc: 0.9885
Epoch 4/20
10/10 [==============================] - 0s 50ms/step - loss: 0.0639 - acc: 0.9951
Epoch 5/20
10/10 [==============================] - 0s 49ms/step - loss: 0.0516 - acc: 0.9959
Epoch 6/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0318 - acc: 0.9979
Epoch 7/20
10/10 [==============================] - 0s 50ms/step - loss: 0.0190 - acc: 0.9986
Epoch

10/10 [==============================] - 12s 1s/step - loss: 0.6393 - acc: 0.8871
Epoch 2/20
10/10 [==============================] - 1s 52ms/step - loss: 0.1199 - acc: 0.9826
Epoch 3/20
10/10 [==============================] - 0s 47ms/step - loss: 0.0109 - acc: 0.9973
Epoch 4/20
10/10 [==============================] - 0s 48ms/step - loss: 0.0068 - acc: 0.9984
Epoch 5/20
10/10 [==============================] - 0s 50ms/step - loss: 0.0014 - acc: 0.9994
Epoch 6/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0041 - acc: 0.9996
Epoch 7/20
10/10 [==============================] - 0s 50ms/step - loss: 0.0036 - acc: 0.9996
Epoch 8/20
10/10 [==============================] - 0s 48ms/step - loss: 5.7557e-04 - acc: 0.9996
Epoch 9/20
10/10 [==============================] - 1s 50ms/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 10/20
10/10 [==============================] - 1s 52ms/step - loss: 1.4104e-04 - acc: 1.0000
Epoch 11/20
10/10 [==============================] - 1s 50m

In [15]:
from numpy import save

In [16]:
save('FO6_f5.npy', p5_f5)


In [17]:
def person_extractor_L(personID):
    p_dataset=[]
    count=0
    for i in range(5,10):
        count = count +1
        if personID==0 and i==0:
            CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',personID,i,True)
        else:
            CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',personID,i,False)
        if count==1 :
            p_dataset = CNN_out_total #done because of need of same dim. for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        print(str(i) + 'word')
    print('----------------')
    print(count)
    print(p_dataset.shape)
    return p_dataset 

In [18]:
p5_l5=person_extractor_L(4)
save('FO6_l5.npy', p5_l5)

Extraction completed 10
Epoch 1/20
10/10 [==============================] - 11s 1s/step - loss: 4.4808 - acc: 0.6738
Epoch 2/20
10/10 [==============================] - 0s 49ms/step - loss: 2.9233 - acc: 0.8025
Epoch 3/20
10/10 [==============================] - 0s 46ms/step - loss: 2.0825 - acc: 0.8627
Epoch 4/20
10/10 [==============================] - 0s 48ms/step - loss: 1.6611 - acc: 0.8928
Epoch 5/20
10/10 [==============================] - 0s 48ms/step - loss: 1.3603 - acc: 0.9133
Epoch 6/20
10/10 [==============================] - 0s 49ms/step - loss: 1.1799 - acc: 0.9244
Epoch 7/20
10/10 [==============================] - 1s 50ms/step - loss: 1.0119 - acc: 0.9350
Epoch 8/20
10/10 [==============================] - 1s 50ms/step - loss: 0.9247 - acc: 0.9408
Epoch 9/20
10/10 [==============================] - 0s 48ms/step - loss: 0.8801 - acc: 0.9437
Epoch 10/20
10/10 [==============================] - 1s 50ms/step - loss: 0.8022 - acc: 0.9486
Epoch 11/20
10/10 [=================

10/10 [==============================] - 0s 50ms/step - loss: 1.1965 - acc: 0.9209
Epoch 5/20
10/10 [==============================] - 1s 50ms/step - loss: 0.8566 - acc: 0.9441
Epoch 6/20
10/10 [==============================] - 1s 52ms/step - loss: 0.6112 - acc: 0.9613
Epoch 7/20
10/10 [==============================] - 0s 48ms/step - loss: 0.5218 - acc: 0.9664
Epoch 8/20
10/10 [==============================] - 0s 50ms/step - loss: 0.4908 - acc: 0.9686
Epoch 9/20
10/10 [==============================] - 0s 49ms/step - loss: 0.4534 - acc: 0.9703
Epoch 10/20
10/10 [==============================] - 0s 50ms/step - loss: 0.4464 - acc: 0.9717
Epoch 11/20
10/10 [==============================] - 0s 49ms/step - loss: 0.4424 - acc: 0.9721
Epoch 12/20
10/10 [==============================] - 1s 50ms/step - loss: 0.4468 - acc: 0.9715
Epoch 13/20
10/10 [==============================] - 0s 50ms/step - loss: 0.4393 - acc: 0.9725
Epoch 14/20
10/10 [==============================] - 1s 50ms/step -

10/10 [==============================] - 0s 50ms/step - loss: 1.2907 - acc: 0.9184
Epoch 9/20
10/10 [==============================] - 1s 50ms/step - loss: 1.2370 - acc: 0.9219
Epoch 10/20
10/10 [==============================] - 0s 48ms/step - loss: 1.1410 - acc: 0.9279
Epoch 11/20
10/10 [==============================] - 0s 49ms/step - loss: 1.0937 - acc: 0.9311
Epoch 12/20
10/10 [==============================] - 0s 48ms/step - loss: 1.0664 - acc: 0.9326
Epoch 13/20
10/10 [==============================] - 1s 50ms/step - loss: 0.9942 - acc: 0.9375
Epoch 14/20
10/10 [==============================] - 0s 49ms/step - loss: 0.9414 - acc: 0.9402
Epoch 15/20
10/10 [==============================] - 0s 48ms/step - loss: 0.8846 - acc: 0.9443
Epoch 16/20
10/10 [==============================] - 0s 48ms/step - loss: 0.8938 - acc: 0.9437
Epoch 17/20
10/10 [==============================] - 0s 50ms/step - loss: 0.9040 - acc: 0.9432
Epoch 18/20
10/10 [==============================] - 1s 52ms/st

10/10 [==============================] - 0s 50ms/step - loss: 0.0410 - acc: 0.9971
Epoch 13/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0451 - acc: 0.9967
Epoch 14/20
10/10 [==============================] - 0s 50ms/step - loss: 0.0343 - acc: 0.9979
Epoch 15/20
10/10 [==============================] - 0s 48ms/step - loss: 0.0343 - acc: 0.9979
Epoch 16/20
10/10 [==============================] - 0s 48ms/step - loss: 0.0406 - acc: 0.9975
Epoch 17/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0376 - acc: 0.9975
Epoch 18/20
10/10 [==============================] - 0s 48ms/step - loss: 0.0347 - acc: 0.9977
Epoch 19/20
10/10 [==============================] - 0s 48ms/step - loss: 0.0348 - acc: 0.9977
Epoch 20/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0343 - acc: 0.9979
Prediction done
(10, 512)
12timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 14s 1s/step - loss: 1.6529 - acc: 0.8156
Epo

10/10 [==============================] - 1s 51ms/step - loss: 2.9886e-04 - acc: 0.9998
Epoch 15/20
10/10 [==============================] - 0s 48ms/step - loss: 2.8104e-04 - acc: 0.9998
Epoch 16/20
10/10 [==============================] - 0s 49ms/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 17/20
10/10 [==============================] - 1s 50ms/step - loss: 9.0488e-04 - acc: 0.9996
Epoch 18/20
10/10 [==============================] - 1s 51ms/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 19/20
10/10 [==============================] - 1s 50ms/step - loss: 1.2177e-04 - acc: 1.0000
Epoch 20/20
10/10 [==============================] - 1s 52ms/step - loss: 3.6746e-04 - acc: 0.9998
Prediction done
(10, 512)
16timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 14s 1s/step - loss: 1.2639 - acc: 0.8305
Epoch 2/20
10/10 [==============================] - 0s 47ms/step - loss: 0.4024 - acc: 0.9588
Epoch 3/20
10/10 [==============================] - 0s 48ms/step - loss

...

10/10 [==============================] - 1s 52ms/step - loss: 2.1273 - acc: 0.8641
Epoch 5/20
10/10 [==============================] - 1s 52ms/step - loss: 1.8465 - acc: 0.8824
Epoch 6/20
10/10 [==============================] - 1s 52ms/step - loss: 1.6748 - acc: 0.8937
Epoch 7/20
10/10 [==============================] - 1s 53ms/step - loss: 1.6041 - acc: 0.8973
Epoch 8/20
10/10 [==============================] - 1s 53ms/step - loss: 1.3688 - acc: 0.9131
Epoch 9/20
10/10 [==============================] - 1s 52ms/step - loss: 1.3402 - acc: 0.9154
Epoch 10/20
10/10 [==============================] - 1s 52ms/step - loss: 1.2244 - acc: 0.9229
Epoch 11/20
10/10 [==============================] - 1s 53ms/step - loss: 1.1403 - acc: 0.9271
Epoch 12/20
10/10 [==============================] - 1s 53ms/step - loss: 1.1000 - acc: 0.9307
Epoch 13/20
10/10 [==============================] - 1s 51ms/step - loss: 1.0002 - acc: 0.9371
Epoch 14/20
10/10 [==============================] - 0s 50ms/step -

10/10 [==============================] - 1s 53ms/step - loss: 1.0124 - acc: 0.9350
Epoch 9/20
10/10 [==============================] - 1s 54ms/step - loss: 0.9510 - acc: 0.9398
Epoch 10/20
10/10 [==============================] - 1s 53ms/step - loss: 0.9677 - acc: 0.9381
Epoch 11/20
10/10 [==============================] - 1s 53ms/step - loss: 0.8634 - acc: 0.9455
Epoch 12/20
10/10 [==============================] - 1s 52ms/step - loss: 0.8290 - acc: 0.9477
Epoch 13/20
10/10 [==============================] - 1s 54ms/step - loss: 0.8254 - acc: 0.9480
Epoch 14/20
10/10 [==============================] - 1s 56ms/step - loss: 0.7974 - acc: 0.9498
Epoch 15/20
10/10 [==============================] - 1s 54ms/step - loss: 0.7477 - acc: 0.9529
Epoch 16/20
10/10 [==============================] - 1s 56ms/step - loss: 0.7017 - acc: 0.9559
Epoch 17/20
10/10 [==============================] - 1s 56ms/step - loss: 0.7051 - acc: 0.9551
Epoch 18/20
10/10 [==============================] - 1s 55ms/st

10/10 [==============================] - 1s 53ms/step - loss: 0.2340 - acc: 0.9852
Epoch 13/20
10/10 [==============================] - 1s 52ms/step - loss: 0.2216 - acc: 0.9857
Epoch 14/20
10/10 [==============================] - 1s 52ms/step - loss: 0.2225 - acc: 0.9855
Epoch 15/20
10/10 [==============================] - 1s 55ms/step - loss: 0.2214 - acc: 0.9859
Epoch 16/20
10/10 [==============================] - 1s 53ms/step - loss: 0.2151 - acc: 0.9863
Epoch 17/20
10/10 [==============================] - 1s 54ms/step - loss: 0.2030 - acc: 0.9869
Epoch 18/20
10/10 [==============================] - 1s 53ms/step - loss: 0.1901 - acc: 0.9879
Epoch 19/20
10/10 [==============================] - 1s 52ms/step - loss: 0.1899 - acc: 0.9881
Epoch 20/20
10/10 [==============================] - 1s 55ms/step - loss: 0.1939 - acc: 0.9875
Prediction done
(10, 512)
11timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 36s 4s/step - loss: 1.5652 - acc: 0.8094
Epo

10/10 [==============================] - 1s 53ms/step - loss: 0.0376 - acc: 0.9975
Epoch 17/20
10/10 [==============================] - 1s 54ms/step - loss: 0.0350 - acc: 0.9975
Epoch 18/20
10/10 [==============================] - 1s 53ms/step - loss: 0.0347 - acc: 0.9975
Epoch 19/20
10/10 [==============================] - 1s 54ms/step - loss: 0.0314 - acc: 0.9979
Epoch 20/20
10/10 [==============================] - 1s 55ms/step - loss: 0.0346 - acc: 0.9977
Prediction done
(10, 512)
15timestep
Extraction completed 10
Epoch 1/20
10/10 [==============================] - 38s 4s/step - loss: 0.8201 - acc: 0.8691
Epoch 2/20
10/10 [==============================] - 1s 51ms/step - loss: 0.1694 - acc: 0.9822
Epoch 3/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0780 - acc: 0.9939
Epoch 4/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0629 - acc: 0.9957
Epoch 5/20
10/10 [==============================] - 1s 50ms/step - loss: 0.0802 - acc: 0.9943
Epoch 6

10/10 [==============================] - 1s 56ms/step - loss: 0.0623 - acc: 0.9961
Epoch 20/20
10/10 [==============================] - 1s 56ms/step - loss: 0.0654 - acc: 0.9959
Prediction done
(10, 512)
19timestep
Concatenation completed
9word
----------------
5
(1000, 512)


In [19]:
#save('FO1_l5.npy', p1_l5)

In [20]:
#CNN_out_test=feature_extractor('C:\\Users\\Om\\Desktop\\Dataset',0,0,0,True)
#CNN_out_total= per_to_feat('C:\\Users\\Om\\Desktop\\Dataset',0,0,True)
#people=5
#p_first5_dataset=[]
#fcount=0
#for i in range(people):
 #   fcount= fcount+1
  #  p_dataset=person_extractor(i)
   # p_dataset_LSTM=reshape_LSTM(p_dataset)
    #if fcount==1:
     #   p_first5_dataset=p_dataset_LSTM
    #se:
    #    p_first5_dataset=np.concatenate((p_first5_dataset,p_dataset_LSTM),axis=0)
        
   # print('person done')